# Search Optimal Haplotype Pair by Lasso

In [1]:
versioninfo()

Julia Version 1.4.2
Commit 44fa15b150* (2020-05-23 18:35 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin18.7.0)
  CPU: Intel(R) Core(TM) i7-6920HQ CPU @ 2.90GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-8.0.1 (ORCJIT, skylake)
Environment:
  JULIA_EDITOR = code
  JULIA_NUM_THREADS = 4


In [2]:
using BenchmarkTools, DataFrames, FileIO, JLD2, LinearAlgebra, Random, Revise

## Test data

Ben prepared the `M` and `N` matrices at window sizes 64, 128, ..., 1024. Let's get the first pair for testing.

In [3]:
@load "example_data.jld2" Ms Ns;
# Ms[1]       # M for width = 64     (M is upper triangular)
# Ms[2]       # M for width = 128   (M is upper triangular) 
# Ns[1]        # N for width = 64 ... etc 

# width = 64 averages 873 haplotype per window. Total win =2789
# width = 128 averages 2497 haplotype per window. Total win = 1394
# width = 256 averages 6985 haplotype per window. Total win =697
# width = 512 averages 17234 haplotype per window. Total win = 348
# width = 1024 averages 32428 haplotype per window. Total win = 174

In [4]:
M = Ms[1]
LinearAlgebra.copytri!(M, 'U')

2344×2344 Array{Float32,2}:
 68.0  58.0  51.0  46.0  45.0  61.0  …  40.0  50.0  59.0  70.0  59.0  53.0
 58.0  76.0  45.0  52.0  53.0  59.0     48.0  56.0  65.0  62.0  65.0  59.0
 51.0  45.0  48.0  41.0  40.0  46.0     35.0  43.0  48.0  53.0  48.0  44.0
 46.0  52.0  41.0  52.0  47.0  47.0     42.0  42.0  51.0  50.0  49.0  47.0
 45.0  53.0  40.0  47.0  48.0  46.0     41.0  41.0  50.0  49.0  48.0  48.0
 61.0  59.0  46.0  47.0  46.0  72.0  …  41.0  49.0  60.0  73.0  58.0  54.0
 55.0  53.0  48.0  47.0  46.0  52.0     41.0  45.0  56.0  57.0  56.0  48.0
 45.0  55.0  38.0  45.0  44.0  46.0     43.0  43.0  56.0  49.0  54.0  46.0
 52.0  50.0  47.0  46.0  45.0  49.0     38.0  44.0  53.0  54.0  53.0  45.0
 45.0  51.0  40.0  47.0  46.0  46.0     39.0  41.0  50.0  49.0  48.0  46.0
 24.0  30.0  17.0  18.0  17.0  25.0  …  18.0  24.0  33.0  28.0  31.0  23.0
 29.0  29.0  22.0  21.0  20.0  28.0     17.0  25.0  30.0  33.0  28.0  26.0
 68.0  74.0  55.0  52.0  51.0  63.0     46.0  62.0  69.0  70.0  71.0  57

In [5]:
N = Ns[1]

1000×2344 Array{Float32,2}:
 52.7333   53.1596  37.5737   33.1596   …  59.1596   46.7333  38.0
 66.0      44.0     42.0      32.1734      62.0      44.0     40.0
 18.0      26.0     16.0      26.0         18.0      22.0     20.0
 26.0      30.0     24.0      30.0         26.0      24.0     26.0
 40.0      38.0     35.1655   37.1655      40.0      36.0     33.1655
 33.925    47.925   28.0      38.0      …  33.925    36.0     41.925
 24.0      24.0     24.0      22.0         22.0      26.0     18.0
 30.0      26.0     22.0      22.0         38.0      22.0     24.0
 47.0691   47.0691  31.0691   33.0691      57.0691   46.0     37.0691
  7.34686  15.3469   3.34686   3.34686      7.34686  15.3469   7.34686
 48.0      38.0     34.0      30.1715   …  56.0      36.0     34.0
 16.0      28.0     12.0      18.0         16.0      26.0     18.0
  4.0      12.0      0.0       0.0          4.0      12.0      4.0
  ⋮                                     ⋱                     
 11.2424   14.9748   9.242

## Current code

This is current code for searching the optimal haplotype pair that minimizes
$$
\|\mathbf{x} - \mathbf{h}_1 - \mathbf{h}_2\|_2^2.
$$

In [6]:
"""
    haplopair!(happair, hapscore, M, N)

Calculate the best pair of haplotypes pairs in the filtered haplotype panel
for each individual in `X` using sufficient statistics `M` and `N`. 

# Note
The best haplotype pairs are column indices of the filtered haplotype panels. 

# Input
* `happair`: optimal haplotype pair for each individual.
* `hapmin`: minimum offered by the optimal haplotype pair.
* `M`: `d x d` matrix with entries `M[i, j] = 2dot(H[:, i], H[:, j]) +
    sumabs2(H[:, i]) + sumabs2(H[:, j])`, where `H` is the haplotype matrix
    with haplotypes in columns. Only the upper triangular part of `M` is used.
* `N`: `n x d` matrix `2X'H`, where `X` is the genotype matrix with individuals
    in columns.
"""
function haplopair!(
    happair1::AbstractVector{<:Integer},
    happair2::AbstractVector{<:Integer},
    hapmin::AbstractVector{T},
    M::AbstractMatrix{T},
    N::AbstractMatrix{T},
    ) where T <: Real
    n, d = size(N)
    fill!(hapmin, typemax(T))
    @inbounds for k in 1:d, j in 1:k
        Mjk = M[j, k]
        # loop over individuals
        for i in 1:n
            score = Mjk - N[i, j] - N[i, k]
            # keep best happair (original code)
            if score < hapmin[i]
                hapmin[i]   = score
                happair1[i] = j
                happair2[i] = k
            end

        end
    end
    return nothing
end

haplopair!

In [7]:
n, d     = size(N)
hapmin   = Vector{Float32}(undef, n)
happair1 = Vector{Int}(undef, n)
happair2 = Vector{Int}(undef, n)

@benchmark haplopair!($happair1, $happair2, $hapmin, $M, $N)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     2.163 s (0.00% GC)
  median time:      2.203 s (0.00% GC)
  mean time:        2.194 s (0.00% GC)
  maximum time:     2.216 s (0.00% GC)
  --------------
  samples:          3
  evals/sample:     1

In [8]:
# this is global optimal solution
DataFrame(hap1 = happair1, hap2 = happair2, score = hapmin)

,hap1,hap2,score
,Int64,Int64,Float32
1,41,1528,-58.8929
2,1,38,-65.0
3,4,11,-18.0
4,10,1277,-29.0
5,577,1207,-39.1655
6,18,25,-47.0
7,50,101,-22.0
8,41,50,-28.0
9,95,207,-54.1382


## Stepwise search heuristics

Let's consider minimizing the loss
$$
f(\boldsymbol{\beta}) = \frac 12 \|\mathbf{x} - \mathbf{H} \boldsymbol{\beta}\|_2^2,
$$
where columns of $\mathbf{H} \in \mathbb{R}^{w \times d}$ are candidate haplotypes. Original problem is solving this problem subject to the constraint that only two $\beta_j$ take values 1 and all others take values 0. We see some heuristics that might circumvent the expensive $O(d^2)$ search.

The gradient of objective is
$$
\nabla f(\boldsymbol{\beta}) = - \mathbf{H}^T (\mathbf{x} - \mathbf{H} \boldsymbol{\beta}) = - \mathbf{H}^T \mathbf{x} + \mathbf{H}^T \mathbf{H} \boldsymbol{\beta}.
$$
Note $\mathbf{H}^T \mathbf{x}$ and $\mathbf{H}^T \mathbf{H}$ are pre-computed and cached. When no haplotypes are in the model, $\boldsymbol{\beta} = \mathbf{0}$. We need to search the maximal element of the gradient vector 
$$
i_1 = \arg \max_i (\mathbf{H}^T \mathbf{x})_i.
$$
Given $h_{i_1}$, we search the second haplotype using the original criterion directly. 

Overall this search heuristic costs $3d$ flops.

In [9]:
Nt = Matrix(transpose(N)) # Nt for column-major access

function haplopair_stepwise!(
    happair1 :: AbstractVector{<:Integer},
    happair2 :: AbstractVector{<:Integer},
    hapmin   :: AbstractVector{T},
    M        :: AbstractMatrix{T}, # d x d
    Nt       :: AbstractMatrix{T}, # d x n
    ) where T <: Real
    d, n = size(Nt)
    fill!(hapmin, typemax(T))
    @inbounds for k in 1:n
        # find the first haplotype
        gmax, i1 = Nt[1, k], 1
        for i in 2:d
            g = Nt[i, k]
            if g > gmax
                gmax = g
                i1   = i
            end
        end
        happair1[k] = i1
        # find the optimal second haplotype given i1 using LS criterion
        for i in 1:d
            score = M[i, i1] - Nt[i, k]
            if score < hapmin[k]
                hapmin[k] = score
                happair2[k] = i
            end
        end
        hapmin[k] -= Nt[i1, k]
    end
    return nothing
end

haplopair_stepwise! (generic function with 1 method)

In [10]:
@benchmark haplopair_stepwise!($happair1, $happair2, $hapmin, $M, $Nt)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     4.231 ms (0.00% GC)
  median time:      4.809 ms (0.00% GC)
  mean time:        4.816 ms (0.00% GC)
  maximum time:     6.203 ms (0.00% GC)
  --------------
  samples:          1037
  evals/sample:     1

Not surprisingly, the heuristic solution is not the global optimal. But we do see some haplotypes from the heuristic approach overlap with the optimal ones. This motivates us to the next heuristic method.

In [11]:
DataFrame(hap1 = happair1, hap2 = happair2, score = hapmin)

,hap1,hap2,score
,Int64,Int64,Float32
1,1473,1051,-56.8929
2,1234,38,-64.1734
3,86,75,-18.0
4,106,886,-28.0
5,2162,88,-35.1655
6,1503,1485,-47.0
7,1016,50,-14.0
8,340,119,-28.0
9,56,505,-51.1382


## Top $r$ pool heuristic method

If we are willing to spend more computing time, we search for the top $r$ haplotypes with largest gradient. Then do exhaustive search within this pool.

In [12]:
function haplopair_topr!(
    happair1 :: AbstractVector{<:Integer},
    happair2 :: AbstractVector{<:Integer},
    hapmin   :: AbstractVector{T},
    M        :: AbstractMatrix{T}, # d x d
    Nt       :: AbstractMatrix{T}, # d x n
    r        :: Integer = 5
    ) where T <: Real
    d, n = size(Nt)
    fill!(hapmin, typemax(T))
    idx = Vector{Int}(undef, d)
    @inbounds for k in 1:n
        # find the top r haplotypes
        @views sortperm!(idx, Nt[:, k]; alg = PartialQuickSort(r), rev = true)
        # for each top haplotype, find the optimal second one
        for riter in 1:r
            i1 = idx[riter]
            for i in 1:d
                score = M[i, i1] - Nt[i1, k] - Nt[i, k]
                if score < hapmin[k]
                    hapmin[k] = score
                    happair1[k] = i1
                    happair2[k] = i
                end
            end
        end
    end
    return nothing
end

haplopair_topr! (generic function with 2 methods)

In [13]:
@benchmark haplopair_topr!($happair1, $happair2, $hapmin, $M, $Nt, 5)

BenchmarkTools.Trial: 
  memory estimate:  112.14 KiB
  allocs estimate:  4002
  --------------
  minimum time:     42.518 ms (0.00% GC)
  median time:      43.379 ms (0.00% GC)
  mean time:        44.396 ms (0.00% GC)
  maximum time:     52.379 ms (0.00% GC)
  --------------
  samples:          113
  evals/sample:     1

Now we see we get more global solutions.

In [14]:
DataFrame(hap1 = happair1, hap2 = happair2, score = hapmin)

,hap1,hap2,score
,Int64,Int64,Float32
1,1820,709,-58.8929
2,1,38,-65.0
3,86,75,-18.0
4,218,1486,-29.0
5,7,1533,-38.1655
6,1503,1485,-47.0
7,101,50,-22.0
8,340,119,-28.0
9,325,517,-53.1382
